# Pipeline 2: Advanced Feature Engineering and Preprocessing

This notebook implements a second preprocessing pipeline that focuses on:
1. **Feature Engineering**: Creating new meaningful features from existing ones
2. **Advanced Scaling**: Using RobustScaler and QuantileTransformer for better handling of outliers
3. **Target Encoding**: Using TargetEncoder for high-cardinality categorical features
4. **Advanced Imputation**: Using IterativeImputer for sophisticated missing value handling
5. **Feature Selection**: Reducing dimensionality while preserving important information

## Motivation

This pipeline differs from a basic approach by:
- Creating domain-specific features that capture relationships between variables
- Using more robust scaling methods that handle outliers better
- Leveraging target information for categorical encoding
- Implementing sophisticated missing value imputation
- Reducing feature space through intelligent selection


In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Enable experimental features FIRST
from sklearn.experimental import enable_iterative_imputer

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import RobustScaler, QuantileTransformer, OneHotEncoder, StandardScaler
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import TargetEncoder
import warnings
warnings.filterwarnings('ignore')

# Set random state for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


In [ ]:
# Load and explore data
data = pd.read_csv('health_insurance_train.csv')
print("Dataset shape:", data.shape)
print("\nFirst few rows:")
print(data.head())
print("\nData types:")
print(data.dtypes)
print("\nMissing values:")
print(data.isnull().sum())
print("\nTarget variable (whrswk) statistics:")
print(data['whrswk'].describe())


In [ ]:
# Separate features and target
X = data.drop('whrswk', axis=1)
y = data['whrswk']

# Identify feature types
categorical_features = ['hhi', 'whi', 'hhi2', 'education', 'race', 'hispanic', 'region']
numerical_features = ['experience', 'kidslt6', 'kids618', 'husby']

print("Categorical features:", categorical_features)
print("Numerical features:", numerical_features)
print("\nCategorical feature value counts:")
for col in categorical_features:
    print(f"\n{col}:")
    print(X[col].value_counts(dropna=False))


In [ ]:
# Feature Engineering Functions
def create_engineered_features(X):
    """
    Create new features based on domain knowledge and relationships between variables
    """
    X_eng = X.copy()
    
    # 1. Family structure features
    X_eng['total_kids'] = X_eng['kidslt6'] + X_eng['kids618']
    X_eng['has_kids'] = (X_eng['total_kids'] > 0).astype(int)
    X_eng['has_young_kids'] = (X_eng['kidslt6'] > 0).astype(int)
    
    # 2. Work-life balance indicators
    X_eng['husby_per_kid'] = X_eng['husby'] / (X_eng['total_kids'] + 1)  # +1 to avoid division by zero
    X_eng['experience_per_kid'] = X_eng['experience'] / (X_eng['total_kids'] + 1)
    
    # 3. Insurance coverage combinations
    X_eng['insurance_coverage'] = (
        (X_eng['hhi'] == 'yes').astype(int) + 
        (X_eng['whi'] == 'yes').astype(int) + 
        (X_eng['hhi2'] == 'yes').astype(int)
    )
    
    # 4. Education level encoding (ordinal)
    education_mapping = {
        '9-11years': 1,
        '12years': 2, 
        '13-15years': 3,
        '16years': 4,
        '>16years': 5
    }
    X_eng['education_encoded'] = X_eng['education'].map(education_mapping)
    
    # 5. Regional economic indicators (based on common knowledge)
    region_economic = {
        'northeast': 4,  # Generally higher income
        'northcentral': 3,
        'south': 2,      # Generally lower income
        'west': 3,
        'other': 2
    }
    X_eng['region_economic'] = X_eng['region'].map(region_economic)
    
    # 6. Experience categories
    X_eng['experience_category'] = pd.cut(
        X_eng['experience'], 
        bins=[0, 5, 15, 25, 100], 
        labels=['entry', 'mid', 'senior', 'expert']
    )
    
    # 7. Interaction features
    X_eng['experience_education'] = X_eng['experience'] * X_eng['education_encoded']
    X_eng['husby_education'] = X_eng['husby'] * X_eng['education_encoded']
    
    return X_eng

# Apply feature engineering
X_engineered = create_engineered_features(X)
print("Original features:", X.shape[1])
print("Engineered features:", X_engineered.shape[1])
print("\nNew features created:")
new_features = [col for col in X_engineered.columns if col not in X.columns]
print(new_features)


In [ ]:
# Update feature lists after engineering
categorical_features_eng = ['hhi', 'whi', 'hhi2', 'education', 'race', 'hispanic', 'region', 'experience_category']
numerical_features_eng = ['experience', 'kidslt6', 'kids618', 'husby', 'total_kids', 'has_kids', 'has_young_kids',
                         'husby_per_kid', 'experience_per_kid', 'insurance_coverage', 'education_encoded',
                         'region_economic', 'experience_education', 'husby_education']

print("Updated categorical features:", categorical_features_eng)
print("Updated numerical features:", numerical_features_eng)
print(f"Total features: {len(categorical_features_eng) + len(numerical_features_eng)}")

# Check for any missing values in engineered features
print("\nMissing values in engineered features:")
print(X_engineered[numerical_features_eng].isnull().sum())


In [ ]:
# Create the advanced preprocessing pipeline
def create_advanced_pipeline():
    """
    Create an advanced preprocessing pipeline with:
    - IterativeImputer for sophisticated missing value handling
    - OneHotEncoder for categorical features (TargetEncoder doesn't work with multiclass targets)
    - RobustScaler for numerical features (robust to outliers)
    - QuantileTransformer for non-linear transformation
    - Feature selection to reduce dimensionality
    """
    
    # Numerical preprocessing pipeline
    numerical_pipeline = Pipeline([
        ('imputer', IterativeImputer(random_state=RANDOM_STATE, max_iter=10)),
        ('scaler', RobustScaler()),  # More robust to outliers than StandardScaler
        ('quantile', QuantileTransformer(output_distribution='normal', random_state=RANDOM_STATE))
    ])
    
    # Categorical preprocessing pipeline
    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    
    # Combine preprocessing for different feature types
    preprocessor = ColumnTransformer([
        ('num', numerical_pipeline, numerical_features_eng),
        ('cat', categorical_pipeline, categorical_features_eng)
    ])
    
    # Full pipeline with feature selection
    full_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('feature_selection', SelectKBest(score_func=mutual_info_regression, k=15)),  # Select top 15 features
        ('regressor', RandomForestRegressor(random_state=RANDOM_STATE, n_estimators=100))
    ])
    
    return full_pipeline

# Create the pipeline
advanced_pipeline = create_advanced_pipeline()
print("Advanced pipeline created successfully!")
print("Pipeline steps:")
for i, (name, step) in enumerate(advanced_pipeline.steps):
    print(f"{i+1}. {name}: {step}")


In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_engineered, y, test_size=0.2, random_state=RANDOM_STATE
)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

# Train the advanced pipeline
print("\nTraining advanced pipeline...")
advanced_pipeline.fit(X_train, y_train)
print("Training completed!")

# Make predictions
y_pred_advanced = advanced_pipeline.predict(X_test)

# Calculate metrics
mae_advanced = mean_absolute_error(y_test, y_pred_advanced)
mse_advanced = mean_squared_error(y_test, y_pred_advanced)
r2_advanced = r2_score(y_test, y_pred_advanced)

print(f"\nAdvanced Pipeline Performance:")
print(f"MAE: {mae_advanced:.4f}")
print(f"MSE: {mse_advanced:.4f}")
print(f"R²: {r2_advanced:.4f}")
print(f"RMSE: {np.sqrt(mse_advanced):.4f}")


In [ ]:
# Compare with dummy regressor (baseline)
# DummyRegressor provides a simple sanity check by comparing against simple rules of thumb
# For regression, we use 'mean' strategy which always predicts the mean of training targets
# This helps us understand if our model is actually learning meaningful patterns

dummy_regressor = DummyRegressor(strategy='mean')
dummy_regressor.fit(X_train, y_train)
y_pred_dummy = dummy_regressor.predict(X_test)

mae_dummy = mean_absolute_error(y_test, y_pred_dummy)
mse_dummy = mean_squared_error(y_test, y_pred_dummy)
r2_dummy = r2_score(y_test, y_pred_dummy)

print(f"\nDummy Regressor (Baseline) Performance:")
print(f"MAE: {mae_dummy:.4f}")
print(f"MSE: {mse_dummy:.4f}")
print(f"R²: {r2_dummy:.4f}")
print(f"RMSE: {np.sqrt(mse_dummy):.4f}")

print(f"\nImprovement over baseline:")
print(f"MAE improvement: {((mae_dummy - mae_advanced) / mae_dummy * 100):.2f}%")
print(f"R² improvement: {((r2_advanced - r2_dummy) / abs(r2_dummy) * 100):.2f}%")

# If our model performs worse than dummy regressor, something is wrong!
if mae_advanced > mae_dummy:
    print("\n⚠️  WARNING: Model performs worse than dummy regressor!")
    print("This suggests the model is not learning meaningful patterns.")
else:
    print("\n✅ Model performs better than dummy regressor - good sign!")


In [ ]:
# Cross-validation to get more robust performance estimate
print("Performing cross-validation...")
cv_scores = cross_val_score(advanced_pipeline, X_engineered, y, cv=5, scoring='neg_mean_absolute_error')
cv_mae = -cv_scores.mean()
cv_std = cv_scores.std()

print(f"Cross-validation MAE: {cv_mae:.4f} (+/- {cv_std * 2:.4f})")
print(f"Individual CV scores: {[-score for score in cv_scores]}")

# This is our estimate for the autograder
print(f"\nEstimated MAE on new data: {cv_mae:.4f}")


In [ ]:
# Hyperparameter Tuning with GridSearchCV
# Following the tips: systematic hyperparameter search with proper parameter ranges
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
import time

print("Starting hyperparameter tuning...")
print("This may take a while due to the comprehensive search...")

# Define parameter grids for each model
# Using logarithmic scales for parameters that can vary widely (as recommended in tips)
param_grids = {
    'KNN': {
        'regressor__n_neighbors': [3, 5, 7, 9, 11, 15],
        'regressor__weights': ['uniform', 'distance'],
        'regressor__p': [1, 2]  # Manhattan vs Euclidean distance
    },
    'SGD': {
        'regressor__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],
        'regressor__learning_rate': ['constant', 'optimal', 'invscaling'],
        'regressor__eta0': [0.01, 0.1, 1.0],
        'regressor__max_iter': [1000, 2000, 5000]
    },
    'Random Forest': {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [None, 10, 20, 30],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__min_samples_leaf': [1, 2, 4]
    },
    'Decision Tree': {
        'regressor__max_depth': [None, 5, 10, 15, 20],
        'regressor__min_samples_split': [2, 5, 10, 20],
        'regressor__min_samples_leaf': [1, 2, 4, 8],
        'regressor__criterion': ['squared_error', 'friedman_mse']
    }
}

# Store results
tuned_models = {}
tuning_results = {}

print("\nTuning models individually...")
for name, param_grid in param_grids.items():
    print(f"\nTuning {name}...")
    start_time = time.time()
    
    # Create base model
    if name == 'KNN':
        base_model = KNeighborsRegressor()
    elif name == 'SGD':
        base_model = SGDRegressor(random_state=RANDOM_STATE, early_stopping=True)
    elif name == 'Random Forest':
        base_model = RandomForestRegressor(random_state=RANDOM_STATE)
    elif name == 'Decision Tree':
        base_model = DecisionTreeRegressor(random_state=RANDOM_STATE)
    
    # Create pipeline with the model
    model_pipeline = Pipeline([
        ('preprocessor', advanced_pipeline.named_steps['preprocessor']),
        ('feature_selection', advanced_pipeline.named_steps['feature_selection']),
        ('regressor', base_model)
    ])
    
    # Grid search with 3-fold CV for speed (5-fold would be better but slower)
    grid_search = GridSearchCV(
        model_pipeline, 
        param_grid, 
        cv=3, 
        scoring='neg_mean_absolute_error',
        n_jobs=-1,  # Use all available cores
        verbose=1
    )
    
    # Fit the grid search
    grid_search.fit(X_train, y_train)
    
    # Store results
    tuned_models[name] = grid_search.best_estimator_
    tuning_results[name] = {
        'best_params': grid_search.best_params_,
        'best_score': -grid_search.best_score_,  # Convert back to positive MAE
        'tuning_time': time.time() - start_time
    }
    
    print(f"{name} tuning completed in {tuning_results[name]['tuning_time']:.2f} seconds")
    print(f"Best MAE: {tuning_results[name]['best_score']:.4f}")

print("\n" + "="*70)
print("HYPERPARAMETER TUNING RESULTS")
print("="*70)
for name, results in tuning_results.items():
    print(f"\n{name}:")
    print(f"  Best MAE: {results['best_score']:.4f}")
    print(f"  Tuning time: {results['tuning_time']:.2f}s")
    print(f"  Best parameters: {results['best_params']}")


In [ ]:
# Evaluate tuned models on test set
print("Evaluating tuned models on test set...")
tuned_test_results = {}

for name, model in tuned_models.items():
    # Make predictions
    y_pred_tuned = model.predict(X_test)
    
    # Calculate metrics
    mae_tuned = mean_absolute_error(y_test, y_pred_tuned)
    mse_tuned = mean_squared_error(y_test, y_pred_tuned)
    r2_tuned = r2_score(y_test, y_pred_tuned)
    
    tuned_test_results[name] = {
        'MAE': mae_tuned,
        'MSE': mse_tuned,
        'R2': r2_tuned,
        'RMSE': np.sqrt(mse_tuned)
    }
    
    print(f"{name} - MAE: {mae_tuned:.4f}, R²: {r2_tuned:.4f}")

# Compare before and after tuning
print("\n" + "="*80)
print("PERFORMANCE COMPARISON: BEFORE vs AFTER TUNING")
print("="*80)
print(f"{'Model':<15} | {'Before MAE':<12} | {'After MAE':<12} | {'Improvement':<12}")
print("-" * 80)

# Get original performance for comparison
original_mae = mae_advanced  # From the advanced pipeline
print(f"{'Advanced Pipeline':<15} | {original_mae:<12.4f} | {mae_advanced:<12.4f} | {'Baseline':<12}")

for name, results in tuned_test_results.items():
    # For comparison, we'll use the original advanced pipeline as baseline
    improvement = ((original_mae - results['MAE']) / original_mae * 100)
    print(f"{name:<15} | {original_mae:<12.4f} | {results['MAE']:<12.4f} | {improvement:>+10.2f}%")

# Find best tuned model
best_tuned_model = min(tuned_test_results.keys(), key=lambda x: tuned_test_results[x]['MAE'])
best_tuned_mae = tuned_test_results[best_tuned_model]['MAE']

print(f"\n🏆 Best tuned model: {best_tuned_model}")
print(f"Best tuned MAE: {best_tuned_mae:.4f}")

if best_tuned_mae < original_mae:
    improvement = ((original_mae - best_tuned_mae) / original_mae * 100)
    print(f"✅ Hyperparameter tuning improved performance by {improvement:.2f}%")
else:
    print("⚠️  Hyperparameter tuning did not improve performance")
    print("This can happen if the original parameters were already good or if we need more data")


In [ ]:
# Training Curves and Convergence Analysis
# Following the tips: visualize SGD convergence and learning curves

print("Creating training curves and convergence analysis...")

# 1. SGD Convergence Analysis
print("\n1. SGD Convergence Analysis")
sgd_model = tuned_models['SGD']
sgd_regressor = sgd_model.named_steps['regressor']

# Get the loss curve from SGD (if available)
if hasattr(sgd_regressor, 'loss_curve_'):
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(sgd_regressor.loss_curve_)
    plt.title('SGD Loss Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid(True)
    
    # Show convergence
    plt.subplot(1, 2, 2)
    # Take last 20% of epochs to show convergence
    start_idx = int(len(sgd_regressor.loss_curve_) * 0.8)
    plt.plot(sgd_regressor.loss_curve_[start_idx:])
    plt.title('SGD Convergence (Last 20% of Epochs)')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"SGD converged after {len(sgd_regressor.loss_curve_)} epochs")
    print(f"Final loss: {sgd_regressor.loss_curve_[-1]:.6f}")
else:
    print("SGD loss curve not available (early_stopping might be enabled)")

# 2. Learning Curves for all models
print("\n2. Learning Curves Analysis")

def plot_learning_curve(model, X, y, title, cv=3):
    """Plot learning curve for a given model"""
    train_sizes = np.linspace(0.1, 1.0, 10)
    train_sizes_abs = (train_sizes * len(X)).astype(int)
    
    train_scores = []
    val_scores = []
    
    for size in train_sizes_abs:
        # Sample data
        indices = np.random.choice(len(X), size, replace=False)
        X_sample = X.iloc[indices] if hasattr(X, 'iloc') else X[indices]
        y_sample = y.iloc[indices] if hasattr(y, 'iloc') else y[indices]
        
        # Cross-validation
        cv_scores = cross_val_score(model, X_sample, y_sample, cv=min(cv, size//10), 
                                  scoring='neg_mean_absolute_error')
        val_scores.append(-cv_scores.mean())
        
        # Training score
        model.fit(X_sample, y_sample)
        train_pred = model.predict(X_sample)
        train_mae = mean_absolute_error(y_sample, train_pred)
        train_scores.append(train_mae)
    
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes_abs, train_scores, 'o-', label='Training MAE', color='blue')
    plt.plot(train_sizes_abs, val_scores, 'o-', label='Validation MAE', color='red')
    plt.title(f'Learning Curve - {title}')
    plt.xlabel('Training Set Size')
    plt.ylabel('MAE')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return train_scores, val_scores

# Plot learning curves for best models
print("Plotting learning curves for best models...")

# Use a subset of data for learning curves (to speed up computation)
X_sample = X_engineered.sample(n=min(2000, len(X_engineered)), random_state=RANDOM_STATE)
y_sample = y[X_sample.index]

# Plot for best tuned model
best_model = tuned_models[best_tuned_model]
plot_learning_curve(best_model, X_sample, y_sample, f'Best Model ({best_tuned_model})')

# Plot for Random Forest (usually shows clear learning curve)
if 'Random Forest' in tuned_models:
    plot_learning_curve(tuned_models['Random Forest'], X_sample, y_sample, 'Random Forest')

print("Learning curves completed!")


In [ ]:
# Update final model selection and autograder submission
print("Updating final model selection based on tuning results...")

# Select the best model (either original advanced pipeline or best tuned model)
if best_tuned_mae < mae_advanced:
    final_model = tuned_models[best_tuned_model]
    final_mae_estimate = best_tuned_mae
    model_type = f"Tuned {best_tuned_model}"
    print(f"✅ Using tuned {best_tuned_model} as final model")
else:
    final_model = advanced_pipeline
    final_mae_estimate = mae_advanced
    model_type = "Advanced Pipeline (Random Forest)"
    print(f"✅ Using original advanced pipeline as final model")

# Retrain on full dataset
print(f"\nRetraining {model_type} on full dataset...")
final_model.fit(X_engineered, y)

# Update cross-validation estimate
print("Performing final cross-validation...")
final_cv_scores = cross_val_score(final_model, X_engineered, y, cv=5, scoring='neg_mean_absolute_error')
final_cv_mae = -final_cv_scores.mean()
final_cv_std = final_cv_scores.std()

print(f"Final CV MAE: {final_cv_mae:.4f} (+/- {final_cv_std * 2:.4f})")
print(f"Individual CV scores: {[-score for score in final_cv_scores]}")

# This is our final estimate for the autograder
print(f"\nFinal estimated MAE on new data: {final_cv_mae:.4f}")


In [ ]:
# Load autograder data and make predictions
print("Loading autograder data...")
data_autograder = pd.read_csv('health_insurance_autograde.csv')
print(f"Autograder data shape: {data_autograder.shape}")

# Apply the same feature engineering to autograder data
X_autograder_eng = create_engineered_features(data_autograder)
print(f"Engineered autograder data shape: {X_autograder_eng.shape}")

# Make predictions on autograder data using final model
print("Making predictions on autograder data...")
predictions_autograder = final_model.predict(X_autograder_eng)
print(f"Predictions shape: {predictions_autograder.shape}")
print(f"Prediction statistics:")
print(f"  Min: {predictions_autograder.min():.4f}")
print(f"  Max: {predictions_autograder.max():.4f}")
print(f"  Mean: {predictions_autograder.mean():.4f}")
print(f"  Std: {predictions_autograder.std():.4f}")

# Prepare submission with final estimates
estimate_MAE_on_new_data = np.array([final_cv_mae])
predictions_autograder_data = predictions_autograder

print(f"\nFinal submission data:")
print(f"Model used: {model_type}")
print(f"Estimated MAE: {estimate_MAE_on_new_data[0]:.4f}")
print(f"Number of predictions: {len(predictions_autograder_data)}")


In [ ]:
# Create final submission file
result = np.append(estimate_MAE_on_new_data, predictions_autograder_data)
pd.DataFrame(result).to_csv("autograder_submission_pipeline2.txt", index=False, header=False)

print("Final submission file 'autograder_submission_pipeline2.txt' created successfully!")
print(f"File contains {len(result)} values (1 MAE estimate + {len(predictions_autograder_data)} predictions)")

# Verify the submission file
submission_check = pd.read_csv("autograder_submission_pipeline2.txt", header=None)
print(f"\nVerification:")
print(f"Submission file shape: {submission_check.shape}")
print(f"First value (MAE estimate): {submission_check.iloc[0, 0]:.4f}")
print(f"Last few predictions: {submission_check.tail().values.flatten()}")

# Summary of improvements made
print(f"\n" + "="*70)
print("PIPELINE 2 IMPROVEMENTS SUMMARY")
print("="*70)
print(f"✅ Feature Engineering: {len(new_features)} new features created")
print(f"✅ Advanced Preprocessing: IterativeImputer + RobustScaler + QuantileTransformer")
print(f"✅ Feature Selection: Mutual information selection (15 best features)")
print(f"✅ Hyperparameter Tuning: GridSearchCV for all 4 models")
print(f"✅ Dummy Baseline: Proper baseline comparison")
print(f"✅ Cross-Validation: 5-fold CV for robust performance estimation")
print(f"✅ Training Curves: SGD convergence and learning curve analysis")
print(f"✅ Final Model: {model_type}")
print(f"✅ Final MAE Estimate: {final_cv_mae:.4f}")
print("="*70)


## Pipeline 2 Analysis and Discussion

### Key Differences from Basic Pipeline

This advanced pipeline differs significantly from a basic approach in several ways:

#### 1. **Feature Engineering**
- **Family Structure Features**: Created `total_kids`, `has_kids`, `has_young_kids` to capture family dynamics
- **Work-Life Balance Indicators**: `husby_per_kid`, `experience_per_kid` to understand work-family balance
- **Insurance Coverage Combinations**: `insurance_coverage` aggregates different insurance types
- **Ordinal Encoding**: `education_encoded` and `region_economic` provide meaningful numerical representations
- **Interaction Features**: `experience_education`, `husby_education` capture multiplicative relationships

#### 2. **Advanced Preprocessing**
- **IterativeImputer**: Uses sophisticated multivariate imputation instead of simple mean/median
- **OneHotEncoder**: Robust categorical encoding with unknown category handling
- **RobustScaler**: More resistant to outliers than StandardScaler (uses median and IQR)
- **QuantileTransformer**: Maps features to normal distribution, helping with non-linear relationships

#### 3. **Feature Selection**
- **Mutual Information**: Uses mutual information regression to select the most informative features
- **Dimensionality Reduction**: Reduces from 22 features to 15 most important ones

#### 4. **Hyperparameter Tuning**
- **GridSearchCV**: Systematic search across parameter spaces for all 4 models
- **Logarithmic Scales**: Proper parameter ranges for parameters that vary widely
- **Cross-Validation**: 3-fold CV during tuning for robust parameter selection

#### 5. **Model Evaluation**
- **Dummy Regressor Baseline**: Proper sanity check against simple rules of thumb
- **Learning Curves**: Visualization of model performance vs training set size
- **SGD Convergence**: Analysis of SGD convergence behavior
- **Performance Comparison**: Before vs after tuning analysis

### How This Pipeline Works

1. **Feature Engineering Phase**: Creates domain-specific features that capture relationships between variables
2. **Preprocessing Phase**: Handles missing values and scales features using robust methods
3. **Encoding Phase**: Uses OneHotEncoder for categorical variables with unknown category handling
4. **Transformation Phase**: Applies quantile transformation to normalize distributions
5. **Selection Phase**: Selects the most informative features using mutual information
6. **Tuning Phase**: Systematic hyperparameter optimization for all models
7. **Evaluation Phase**: Comprehensive evaluation with baselines and learning curves
8. **Final Selection**: Chooses best model based on performance

### Expected Benefits

- **Better Feature Representation**: Engineered features capture domain knowledge
- **Robust to Outliers**: RobustScaler handles extreme values better
- **Systematic Optimization**: GridSearchCV finds optimal hyperparameters
- **Sophisticated Imputation**: IterativeImputer considers relationships between features
- **Dimensionality Reduction**: Feature selection reduces overfitting risk
- **Comprehensive Evaluation**: Multiple evaluation methods ensure model quality
- **Baseline Comparison**: Dummy regressor provides sanity check

This pipeline should perform significantly better than a basic approach, especially in handling the complexity and relationships present in the health insurance dataset.
